In [1]:
import requests
import pickle
import pandas as pd
import json as js

In [19]:
game_data = []
year = '2021'
season_type = '02' 
max_game_ID = 1313

In [20]:
for i in range(650,max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

In [5]:
#ezzel atugorjuk a 0-s id-t, ilyen jatek nincs
#game_data = game_data[1:]

In [21]:
#lista elso elemet vizsgaljuk
game_data[0]

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'gamePk': 2021021312,
 'link': '/api/v1/game/2021021312/feed/live',
 'metaData': {'wait': 10, 'timeStamp': '20220430_052221'},
 'gameData': {'game': {'pk': 2021021312, 'season': '20212022', 'type': 'R'},
  'datetime': {'dateTime': '2022-04-30T02:00:00Z',
   'endDateTime': '2022-04-30T04:25:10Z'},
  'status': {'abstractGameState': 'Final',
   'codedGameState': '7',
   'detailedState': 'Final',
   'statusCode': '7',
   'startTimeTBD': False},
  'teams': {'away': {'id': 28,
    'name': 'San Jose Sharks',
    'link': '/api/v1/teams/28',
    'venue': {'name': 'SAP Center at San Jose',
     'link': '/api/v1/venues/null',
     'city': 'San Jose',
     'timeZone': {'id': 'America/Los_Angeles', 'offset': -7, 'tz': 'PDT'}},
    'abbreviation': 'SJS',
    'triCode': 'SJS',
    'teamName': 'Sharks',
    

In [13]:
len(game_data)

662

In [22]:
players = list()

for idx, i in enumerate(game_data):
    try:
        tmp_players_df = pd.DataFrame(i['gameData']['players']).transpose()
        tmp_players_df = tmp_players_df.assign(gameId = i['gameData']['game']['pk'])
        tmp_players_df = tmp_players_df.assign(teamID = lambda x: x['currentTeam'].apply(lambda y: y['id']))
        tmp_players_df = tmp_players_df.assign(pPos = lambda x: x['primaryPosition'].apply(lambda y: y['name']))
        tmp_players_df = tmp_players_df.drop(columns=['currentTeam', 'primaryPosition'])
        players.append(tmp_players_df)
    except:
        continue

players = pd.concat(players)
players.to_csv('players_df_2021_2.csv', index=False)

players

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,gameId,teamID,pPos
ID8482093,8482093,Seth Jarvis,/api/v1/people/8482093,Seth,Jarvis,24,2002-02-01,20,Winnipeg,MB,...,175,True,False,False,True,R,Y,2021020654,12,Center
ID8480074,8480074,Alexandre Texier,/api/v1/people/8480074,Alexandre,Texier,42,1999-09-13,23,Saint-Martin-d'Hères,NaN,...,186,True,False,False,False,L,Y,2021020654,29,Center
ID8478904,8478904,Steven Lorentz,/api/v1/people/8478904,Steven,Lorentz,78,1996-04-13,26,Kitchener,ON,...,206,True,False,False,False,L,Y,2021020654,12,Center
ID8480829,8480829,Jesperi Kotkaniemi,/api/v1/people/8480829,Jesperi,Kotkaniemi,82,2000-07-06,22,Pori,NaN,...,201,True,False,False,False,L,Y,2021020654,12,Center
ID8473533,8473533,Jordan Staal,/api/v1/people/8473533,Jordan,Staal,11,1988-09-10,34,Thunder Bay,ON,...,220,True,False,True,False,L,Y,2021020654,12,Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476884,8476884,Derrick Pouliot,/api/v1/people/8476884,Derrick,Pouliot,51,1994-01-16,28,Estevan,SK,...,196,True,False,False,False,L,Y,2021021312,55,Defenseman
ID8474586,8474586,Jordan Eberle,/api/v1/people/8474586,Jordan,Eberle,7,1990-05-15,32,Regina,SK,...,187,True,True,False,False,R,Y,2021021312,55,Right Wing
ID8476881,8476881,Tomas Hertl,/api/v1/people/8476881,Tomas,Hertl,48,1993-11-12,28,Praha,NaN,...,215,True,True,False,False,L,Y,2021021312,28,Center
ID8479395,8479395,Dennis Cholowski,/api/v1/people/8479395,Dennis,Cholowski,2,1998-02-15,24,Langley,BC,...,197,True,False,False,False,L,Y,2021021312,55,Defenseman


In [23]:
teams = list()

for i in game_data:
    try:
        tmp_teams_df = pd.DataFrame(i['gameData']['teams']).transpose()
        tmp_teams_df = tmp_teams_df.drop(columns=['venue','division','conference','franchise','officialSiteUrl','franchiseId'])
        teams.append(tmp_teams_df)
    except:
        continue

teams = pd.concat(teams)
teams.to_csv('teams_df_2021_2.csv', index=False)
teams

,id,name,link,abbreviation,triCode,teamName,locationName,firstYearOfPlay,shortName,active
away,4,Philadelphia Flyers,/api/v1/teams/4,PHI,PHI,Flyers,Philadelphia,1967,Philadelphia,True
home,6,Boston Bruins,/api/v1/teams/6,BOS,BOS,Bruins,Boston,1924,Boston,True
away,52,Winnipeg Jets,/api/v1/teams/52,WPG,WPG,Jets,Winnipeg,2011,Winnipeg,True
home,17,Detroit Red Wings,/api/v1/teams/17,DET,DET,Red Wings,Detroit,1932,Detroit,True
away,23,Vancouver Canucks,/api/v1/teams/23,VAN,VAN,Canucks,Vancouver,1970,Vancouver,True
...,...,...,...,...,...,...,...,...,...,...
home,23,Vancouver Canucks,/api/v1/teams/23,VAN,VAN,Canucks,Vancouver,1970,Vancouver,True
away,18,Nashville Predators,/api/v1/teams/18,NSH,NSH,Predators,Nashville,1998,Nashville,True
home,53,Arizona Coyotes,/api/v1/teams/53,ARI,ARI,Coyotes,Arizona,2014,Arizona,True
away,28,San Jose Sharks,/api/v1/teams/28,SJS,SJS,Sharks,San Jose,1991,San Jose,True


In [24]:
games = list()

for idx, i in enumerate(game_data):
    try:
        tmp_games_df = pd.DataFrame(i['gameData']['game'],index=[0])
        tmp_games_df = tmp_games_df.assign(startDate = i['gameData']['datetime']['dateTime'])
        tmp_games_df = tmp_games_df.assign(endDate = i['gameData']['datetime']['endDateTime'])
        tmp_games_df = tmp_games_df.assign(status = i['gameData']['status']['abstractGameState'])
        tmp_games_df = tmp_games_df.assign(teamAwayID = i['gameData']['teams']['away']['id'])
        tmp_games_df = tmp_games_df.assign(teamHomeID = i['gameData']['teams']['home']['id'])
        games.append(tmp_games_df)
    except:
        continue
            
games = pd.concat(games)
games.to_csv('games_df_2021_2.csv', index=False)
    
games

,pk,season,type,startDate,endDate,status,teamAwayID,teamHomeID
0,2021020650,20212022,R,2022-01-14T00:00:00Z,2022-01-14T02:46:07Z,Final,4,6
0,2021020651,20212022,R,2022-01-14T00:30:00Z,2022-01-14T03:02:07Z,Final,52,17
0,2021020652,20212022,R,2022-01-14T00:00:00Z,2022-01-14T02:42:34Z,Final,23,14
0,2021020653,20212022,R,2022-01-14T00:30:00Z,2022-01-14T03:05:28Z,Final,1,2
0,2021020654,20212022,R,2022-01-14T00:00:00Z,2022-01-14T02:24:02Z,Final,29,12
...,...,...,...,...,...,...,...,...
0,2021021308,20212022,R,2022-04-30T00:00:00Z,2022-04-30T02:39:06Z,Final,21,30
0,2021021309,20212022,R,2022-04-30T00:00:00Z,2022-04-30T02:29:54Z,Final,20,52
0,2021021310,20212022,R,2022-04-29T02:00:00Z,2022-04-29T04:33:24Z,Final,26,23
0,2021021311,20212022,R,2022-04-30T02:30:00Z,2022-04-30T05:12:33Z,Final,18,53


In [25]:
events = list()

for idx, i in enumerate(game_data):
    try:
        tmp_events_df = pd.DataFrame(i['liveData']['plays']['allPlays'])
        tmp_events_df = tmp_events_df.assign(gameId = i['gameData']['game']['pk'])
        tmp_events_df = tmp_events_df.assign(eventId = lambda x: x['about'].apply(lambda y: y['eventIdx']))
        tmp_events_df = tmp_events_df.assign(eventType = lambda x: x['result'].apply(lambda y: y['event']))
        tmp_events_df = tmp_events_df.assign(eventTypeDesc = lambda x: x['result'].apply(lambda y: y['description']))
        tmp_events_df = tmp_events_df.assign(period = lambda x: x['about'].apply(lambda y: y['period']))
        tmp_events_df = tmp_events_df.assign(periodTime = lambda x: x['about'].apply(lambda y: y['periodTime']))
        tmp_events_df = tmp_events_df.assign(periodType = lambda x: x['about'].apply(lambda y: y['periodType']))
        tmp_events_df = tmp_events_df.assign(teamAwayID = i['gameData']['teams']['away']['id'])
        tmp_events_df = tmp_events_df.assign(teamHomeID = i['gameData']['teams']['home']['id'])
        tmp_events_df = tmp_events_df.assign(goalsHome = lambda x: x['about'].apply(lambda y: y['goals']['home']))
        tmp_events_df = tmp_events_df.assign(goalsAway = lambda x: x['about'].apply(lambda y: y['goals']['away']))
        tmp_events_df = tmp_events_df.assign(coX = lambda x: x['coordinates'].apply(lambda y: y['x'] if y else ''))
        tmp_events_df = tmp_events_df.assign(coY = lambda x: x['coordinates'].apply(lambda y: y['y'] if y else ''))

        result_player1Id = []
        result_player1Type = []

        for i in tmp_events_df['players']:
            try:
                result_player1Id.append(i[0]['player']['id'])
                result_player1Type.append(i[0]['playerType'])
            except IndexError:
                result_player1Id.append("")
                result_player1Type.append("")
            except TypeError:
                result_player1Id.append('')
                result_player1Type.append("")
            except ValueError:
                result_player1Id.append('')
                result_player1Type.append("")     

        result_player2Id = []
        result_player2Type = []

        for i in tmp_events_df['players']:
            try:
                result_player2Id.append(i[1]['player']['id'])
                result_player2Type.append(i[1]['playerType'])
            except IndexError:
                result_player2Id.append('')
                result_player2Type.append('')
            except TypeError:
                result_player2Id.append('')
                result_player2Type.append('')
            except ValueError:
                result_player2Id.append('')
                result_player2Type.append('')

        result_player3Id = []
        result_player3Type = []

        for i in tmp_events_df['players']:
            try:
                result_player3Id.append(i[2]['player']['id'])
                result_player3Type.append(i[2]['playerType'])
            except IndexError:
                result_player3Id.append('')
                result_player3Type.append('')
            except TypeError:
                result_player3Id.append('')
                result_player3Type.append('')
            except ValueError:
                result_player3Id.append('')
                result_player3Type.append('')

        result_player4Id = []
        result_player4Type = []

        for i in tmp_events_df['players']:
            try:
                result_player4Id.append(i[3]['player']['id'])
                result_player4Type.append(i[3]['playerType'])
            except IndexError:
                result_player4Id.append('')
                result_player4Type.append('')
            except TypeError:
                result_player4Id.append('')
                result_player4Type.append('')
            except ValueError:
                result_player4Id.append('')
                result_player4Type.append('')

        tmp_events_df = tmp_events_df.assign(player1Id = result_player1Id)
        tmp_events_df = tmp_events_df.assign(player1Type = result_player1Type)
        tmp_events_df = tmp_events_df.assign(player2Id = result_player2Id)
        tmp_events_df = tmp_events_df.assign(player2Type = result_player2Type)
        tmp_events_df = tmp_events_df.assign(player3Id = result_player3Id)
        tmp_events_df = tmp_events_df.assign(player3Type = result_player3Type)
        tmp_events_df = tmp_events_df.assign(player4Id = result_player4Id)
        tmp_events_df = tmp_events_df.assign(player4Type = result_player4Type)

        tmp_events_df = tmp_events_df.drop(columns=(['result','about','coordinates','players','team']))
            
        events.append(tmp_events_df)
    except:
        continue
    
events = pd.concat(events)
events.to_csv('events_df_2021_2.csv', index=False)

events

,gameId,eventId,eventType,eventTypeDesc,period,periodTime,periodType,teamAwayID,teamHomeID,goalsHome,...,coX,coY,player1Id,player1Type,player2Id,player2Type,player3Id,player3Type,player4Id,player4Type
0,2021020650,0,Game Scheduled,Game Scheduled,1,00:00,REGULAR,4,6,0,...,,,,,,,,,,
1,2021020650,1,Period Ready,Period Ready,1,00:00,REGULAR,4,6,0,...,,,,,,,,,,
2,2021020650,2,Period Start,Period Start,1,00:00,REGULAR,4,6,0,...,,,,,,,,,,
3,2021020650,3,Faceoff,Tomas Nosek faceoff won against Kevin Hayes,1,00:00,REGULAR,4,6,0,...,0.0,0.0,8477931,Winner,8475763,Loser,,,,
4,2021020650,4,Stoppage,Puck in Netting,1,00:09,REGULAR,4,6,0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2021021312,258,Blocked Shot,Jaycob Megna shot blocked shot by Kole Lind,3,19:54,REGULAR,28,55,3,...,-44.0,-21.0,8479986,Blocker,8477034,Shooter,,,,
259,2021021312,259,Period End,End of 3rd Period,3,20:00,REGULAR,28,55,3,...,,,,,,,,,,
260,2021021312,260,Period Official,Period Official,3,20:00,REGULAR,28,55,3,...,,,,,,,,,,
261,2021021312,261,Game End,Game End,3,20:00,REGULAR,28,55,3,...,,,,,,,,,,
